k-nearest neighbors (kNN) neigt bei unausgeglichenen Daten dazu,  die Mehrheitsklasse zu bevorzugen, da er auf der Nähe zu den nächsten Nachbarn basiert. Dagegen hilft: 

1. **Resampling**: Mehrheitsklasse unterabtasten (Unterabtastung) oder Minderheitsklasse überabtasten (Überabtastung),  z.B. SMOTE (Synthetic Minority Over-sampling Technique) oder Random Oversampling (siehe https://iopscience.iop.org/article/10.1088/1757-899X/719/1/012072/pdf).
    - Ergebnis: SMOTE besser als Random Oversampling

2. **Gewichtete Distanzen**:  Distanzen in der kNN-Berechnung zu gewichten, Anpassung der `weights` Parameter in der `KNeighborsClassifier` Funktion in scikit-learn erreicht werden. (nicht so effektiv) 

3. **Anomalieerkennung**: In einigen Fällen kann es sinnvoll sein, das Problem als Anomalieerkennung zu behandeln, bei der die Minderheitsklasse als "Anomalie" betrachtet wird. zb Local Outlier Factor (LOF) Er berechnet einen Score, der angibt, wie isoliert ein Punkt im Vergleich zu seiner Umgebung ist. 
    - Ergebnis: Anomalieerkennung funktioniert nicht so gut. 


In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier, LocalOutlierFactor
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.metrics import fbeta_score
from sklearn.ensemble import IsolationForest

# Angenommen, Ihre Daten sind in einem DataFrame df
df = pd.read_csv('../data/GeneralDatensatz18-21ohneGeo-mitLockdown_mitCorona.csv', sep=';')

X = df[['UMONAT','USTUNDE','UWOCHENTAG','UART','USTRZUSTAND','BEZ','UTYP1','ULICHTVERH','IstRad','IstPKW','IstFuss','IstKrad','IstGkfz','IstSonstige', 'LOCKDOWN', 'COVID']]
y = df['UKATEGORIE'].isin([1, 2]).astype(int)

# Initialisieren Sie KFold
kf = KFold(n_splits=3, random_state=42, shuffle=True)

# Initialisieren Sie Listen, um die Ergebnisse zu speichern
fbeta_scores_SMOTE = []




# Loop über jeden Split
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Anwendung von SMOTE
    sm = SMOTE(random_state=42)
    X_res, y_res = sm.fit_resample(X_train, y_train)

    # Trainieren Sie den Klassifikator
    knn = KNeighborsClassifier(n_neighbors=3, weights='distance')
    knn.fit(X_res, y_res)

    # Vorhersagen auf den Testdaten machen
    y_pred = knn.predict(X_test)

    # F-beta-Score berechnen und zur Liste hinzufügen
    fbeta = fbeta_score(y_test, y_pred, beta=2)
    fbeta_scores_SMOTE.append(fbeta)

    # Anwendung von Random OverSampling
    ros = RandomOverSampler(random_state=42)
    X_res, y_res = ros.fit_resample(X_train, y_train)

    # Trainieren Sie den Klassifikator
    knn.fit(X_res, y_res)

    # Vorhersagen auf den Testdaten machen
    y_pred = knn.predict(X_test)

    # F-beta-Score berechnen und zur Liste hinzufügen
    fbeta = fbeta_score(y_test, y_pred, beta=2)
    fbeta_scores_ro.append(fbeta)
    
     # Führen Sie den LOF Algorithmus auf den Trainingsdaten aus
    y_pred_train = lof.fit_predict(X_train)

    # Wandle die Vorhersagen in binäre Werte um: -1 wird zu 1 (Anomalie), 1 wird zu 0 (normal)
    y_pred_train_binary = (y_pred_train == -1).astype(int)

    # Führen Sie den LOF Algorithmus auf den Testdaten aus
    y_pred_test = lof.fit_predict(X_test)

    # Wandle die Vorhersagen in binäre Werte um: -1 wird zu 1 (Anomalie), 1 wird zu 0 (normal)
    y_pred_test_binary = (y_pred_test == -1).astype(int)

    # Berechne den F-beta-Score für die Trainingsdaten
    fbeta_train = fbeta_score(y_train, y_pred_train_binary, beta=2)

    # Berechne den F-beta-Score für die Testdaten
    fbeta_test = fbeta_score(y_test, y_pred_test_binary, beta=2)

    # Füge den durchschnittlichen F-beta-Score zur Liste hinzu
    fbeta_scores_LOF.append((fbeta_train + fbeta_test) / 2)


# Durchschnittlichen F-beta-Score über alle Folds ausgeben
print("Average F-beta score for SMOTE:", pd.Series(fbeta_scores_SMOTE).mean())
print("Average F-beta score for random oversampling:", pd.Series(fbeta_scores_ro).mean())
print(f"Average F-beta score for LOF: {pd.Series(fbeta_scores_LOF).mean()}")



Average F-beta score for SMOTE: 0.27417020902581585
Average F-beta score for random oversampling: 0.2401041078791355
Average F-beta score for LOF: 0.11777800006519033


Erklärung LOC-Code: Die Vorhersagen y_pred sind -1 für Anomalien und 1 für normale Datenpunkte. Diese werden dann in ein binäres Format konvertiert (1 für Anomalien, 0 für normale Datenpunkte), das mit den wahren Labels verglichen werden kann. 